In [ ]:
import gpt_wrapper
from gpt_wrapper.chat import Chat

gpt_wrapper.api_base = "http://mnlp-backend-lb-1062233132.eu-central-1.elb.amazonaws.com"
gpt_wrapper.api_key = "6a080da7-d898-45f1-81c7-0148de590a62"

In [ ]:
import json
import copy

# Load your JSON file
with open("/Users/aminebelghmi/Desktop/project-m1-2025-ahya-1/UTF-8311528.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Make a copy to modify
updated_data = copy.deepcopy(data)

# Initialize counter
counter = 0

# Open a log file to record all prompts and responses
log_file = open("generation_log.jsonl", "w", encoding="utf-8")

# Loop through each question
for idx, question in enumerate(updated_data):
    question_body = question.get("question_body", "").strip()
    question_answer = question.get("question_answer", "").strip()
    question_type = question.get("question_type", "").strip()
    question_options = question.get("question_options", None)
    question_id = question.get("question_id", "")
    chat = Chat.create(f"""Question_{question_id}_3""")

    prompt = f"""I am collecting preference pair data for Direct preference optimization fine tuning. I would like you to generate preference pairs for my data and rank them accordingly. The data contains multiple fields. For each question, you should generate two different answers for the A and B fields of the data sample. Importantly, each preference pair should contain one correct answer and one wrong answer. In the case of open questions, a possible correct answer is given in the field : question_answer. In the case of mcq, the correct answer is given in the field question_answer also. So you should always give one good reasoning that leads to the correct answer, and one false reasoning that leads to a false answer. Importantly, the preference pairs should be 1. Similar in quality. The difference in quality between them shouldn’t be so big
that it’d be immediately obvious and possibly counter-productive to teach it to
the model (for example, one has correct grammar and the other doesn’t).
2. Diverse. You should find ways to generate diverse answers to the questions. If
you generate the “bad” answer in the pair by negating the good one, then
you’re just creating an artifact rather than quality data.
3. Sufficiently complex. You don’t want to generate two bad answers and pick
the least bad one, because that is not teaching the model what a good,
aligned answer actually looks like. Your answers should be complex enough to
answer the problem.
The answers should be detailed enough.
here is an example of filled data sample : 
{{
    "course_id": 15000,
    "question_id": 51,
    "question_body": "We have a collection of rectangles in a plane, whose sides are aligned with the coordinate axes. Each rectangle is represented by its lower left corner $$(x_1,y_1)$$ and its upper right corner $$(x_2,y_2)$$. All coordinates are of type Long. We require $$x_1 \\le x_2$$ and $$y_1 \\le y_2$$. How can the result be computed in parallel? Which properties of hull2 need to hold to make the solution correct? Prove these properties for hull2.",
    "question_options": null,
    "question_answer": "For being computed with a parallel reduction, hull2 needs to be associative. In fact, it is associative as well as commutative here. We slightly abuse notation to keep the structure similar to code, with a Rectangle structure.  Given $$ r_i = Rectangle((x_1^i, y_1^i), (x_2^i, y_2^i)) $$  we have,  \\begin{{align*}} &hull2(r_1, hull2(r_2, r_3)) &= hull2(r_1, Rectangle((min(x_1^2, x_1^3), min(y_1^2, y_1^3)), (max(x_2^2, x_2^3), max(y_2^2, y_2^3))))\\  &= Rectangle((min(x_1^1, min(x_1^2, x_1^3)), min(y_1^1, min(y_1^2, y_1^3))), (max(x_2^1, max(x_2^2, x_2^3)), max(y_2^1, max(y_2^2, y_2^3))))\\\\  &= Rectangle((min(x_1^1, x_1^2, x_1^3), min(y_1^1, y_1^2, y_1^3)), (max(x_2^1, x_2^2, x_2^3), max(y_2^1, y_2^2, y_2^3))) \\end{{align*}}  Note that at this step, the order of $$r_1, r_2, r_3$$ does not matter, $$min$$ and $$max$$ are both associative and commutative, so the associativity and commutativity of $$hull2$$ are visible here! We reduce from here to prove associativity, and the same steps are followed for commutativity.  \\begin{{align*}} &hull2(r_1, r_2) &= Rectangle((min(x_1^1, x_1^2), min(y_1^1, y_1^2)), (max(x_2^1, x_2^2), max(y_2^1, y_2^2)))  &= Rectangle((min(x_1^2, x_1^1), min(y_1^2, y_1^1)), (max(x_2^2, x_2^1), max(y_2^2, y_2^1))) \\\\  &= hull2(r_2, r_1) \\end{{align*}}",
    "question_type": "open_answer",
    "A": "For parallel computation using reduction, hull2 must be associative, so that grouping of operations does not affect the result. Associativity ensures that combining rectangles in any order yields the same bounding rectangle. Additionally, hull2 is also commutative since min and max operations used in its definition are commutative. This allows further flexibility in computation ordering, which is key in parallel processing.",
    "B": "To compute in parallel, hull2 only needs to be commutative so that the order of rectangle combination does not matter. Associativity is not necessary because each parallel process will handle independent parts of the data before combining results at the end.",
    "ranking_criteria": {{
        "overall": "A",
        "correctness": "A",
        "relevance": "A",
        "clarity": "A",
        "completeness": "A"
    }}
}} and here is the data sample for which you should generate preference pairs : {question}.
""";
    
    message = chat.ask(content = prompt, instruction = "your answer should only be : Answer1: [the actual first answer, the correct one]   Answer2: [The other actual answer, the false one], you should not output anything apart Answer1: [actual answer, the correct one] Answer2: [other answer, the false one]. Here is an example of what your output should look like exactly. Answer1: I like apples. Answer2: I like bananas.")


    # Parse using substring logic
    response = message.to_dict()['content'].strip()
    answer1 = ""
    answer2 = ""
    
    if "Answer1:" in response and "Answer2:" in response:
        start_1 = response.find("Answer1:") + len("Answer1:")
        start_2 = response.find("Answer2:")
        answer1 = response[start_1:start_2].strip()
        answer2 = response[start_2 + len("Answer2:"):].strip()
    else:
        answer1 = response
        answer2 = response

    updated_data[idx]["A"] = answer1
    updated_data[idx]["B"] = answer2

    # Log the interaction
    log_file.write(json.dumps({
        "question_id": question.get("question_id"),
        "type": question_type,
        "response": response
    }) + "\n")

    counter += 1
    print(f"Generated {counter} preference pairs so far.")

# Save the updated JSON
with open("updated_questions.json", "w", encoding="utf-8") as f:
    json.dump(updated_data, f, indent=4, ensure_ascii=False)

log_file.close()
print("All preference pairs generated and saved to 'updated_questions.json'. Full log in 'generation_log.jsonl'.")


In [ ]:
Chat.budget()